### アイテムベース協調フィルタリング
今回は1）jaccard指数を用いたレコメンドと2）コサイン類似度を用いたレコメンドの2種類で実験

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import pairwise_distances

#### データセットの内容確認

In [ ]:
u_data_org = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
u_data_org.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#### 学習用・テスト用データの取り込み

In [ ]:
# ユーザ×評価値のデータ
u_data_train = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/ua.base', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
u_data_test = pd.read_csv(
   'http://files.grouplens.org/datasets/movielens/ml-100k/ua.test', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")

# 件数の確認
train_cnt = u_data_train.count()
test_cnt = u_data_test.count()
print('Train Set:', str(train_cnt), '\n')
print('Test Set:', str(test_cnt))

Train Set: user_id      90570
item_id      90570
rating       90570
timestamp    90570
dtype: int64 

Test Set: user_id      9430
item_id      9430
rating       9430
timestamp    9430
dtype: int64


#### データをitem_id × user_idの行列へ整形

In [ ]:
# item_id x user_idの行列に変換する
item_list = u_data_org.sort_values('item_id').item_id.unique()
user_list = u_data_org.user_id.unique()
rating_matrix_item = np.zeros([len(item_list), len(user_list)])

for item_id in tqdm(range(1, len(item_list))):
    user_list_item = u_data_train[u_data_train['item_id'] == item_id].sort_values('user_id').user_id.unique()
    for user_id in user_list_item:
        try:
            user_rate = u_data_train[(u_data_train['item_id'] == item_id) & (u_data_train['user_id'] == user_id)].loc[:, 'rating']
        except:
            user_rate = 0
        rating_matrix_item[item_id-1, user_id-1] = user_rate


100%|██████████| 1681/1681 [01:59<00:00, 14.05it/s] 


In [ ]:
rating_matrix_item

array([[5., 4., 0., ..., 5., 0., 0.],
       [3., 0., 0., ..., 0., 0., 5.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# 本当はこんなオブジェクト使わなくてもいけると思うのですがいいやり方が浮かばず…。いい方法あれば教えてください。

# item x userの評価したかどうか{0, 1}がわかる行列作成
rating_matrix_calc = rating_matrix_item.copy()
rating_matrix_calc[rating_matrix_calc != 0] = 1
# 評価していないアイテムに1が立つ行列を作成。後で使う
rating_matrix_train = np.abs(rating_matrix_calc - 1)

### 類似度行列の計算

In [ ]:
# おとなしくpairwise_distanceを使う

# コサイン
similarity_matrix = 1 - pairwise_distances(rating_matrix_item, metric='cosine')

# jaccard
similarity_matrix = 1 - pairwise_distances(rating_matrix_item, metric='jaccard')

# 対角成分の値はゼロにする
np.fill_diagonal(similarity_matrix, 0)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
similarity_matrix

array([[0.        , 0.22142857, 0.17199017, ..., 0.        , 0.00255102,
        0.        ],
       [0.22142857, 0.        , 0.17045455, ..., 0.        , 0.00826446,
        0.        ],
       [0.17199017, 0.17045455, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00255102, 0.00826446, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### 予測評価値の計算・レコメンド

In [ ]:
user_id = 100
hits = 0

# 各ユーザの評価値を抜き出し「類似度×評価点」を算出
rating_matrix_user = rating_matrix_item[:, user_id - 1]
pred_rating_user = similarity_matrix * rating_matrix_user
print(pred_rating_user)
# アイテム（行）ごとに「類似度×評価点」を合計
pred_rating_user = pred_rating_user.sum(axis=1)
print(pred_rating_user)

# スコアの分母の方。ユーザが評価したアイテムの類似度だけを足し合わせる - 数式通りにやろうとしたらうまくいかなかったためこの分母の部分は除外
# sim_user_calc = similarity_matrix * rating_matrix_calc[:, user_id - 1]
# sim_user_calc_base = sim_user_calc.sum(axis=1)

# ユーザが既に評価したアイテムのスコアはゼロに直す
pred_rating_user_item = pred_rating_user * rating_matrix_train[:,user_id - 1]

#ここからレコメンドされたアイテムがどれだけあっていたかを評価していく
recommend_list = np.argsort(pred_rating_user_item)[::-1][:10] + 1
purchase_list_user = u_data_test[u_data_test.user_id == user_id].loc[:, 'item_id'].unique()
for item_id in recommend_list:
    if item_id in purchase_list_user:
        hits += 1
pre = hits / 10.0

print('Recommend list:', recommend_list)
print('Test Rated list:', purchase_list_user)
print('Precision:', str(pre))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[16.92595783 12.15348077 10.89480184 ...  2.53353438  0.17578952
         nan]
Recommend list: [1682 1582 1653  302  307  750  301  748  288  332]
Test Rated list: [266 268 288 302 321 340 344 354 355 750]
Precision: 0.3


#### 全体の精度評価

In [ ]:
# 予測評価値の計算
precision_list = []
recall_list = []
user_list_test = u_data_test.sort_values('user_id').user_id.unique()

for user_id in tqdm(user_list_test):
    hits = 0
    # 各ユーザの評価値を抜き出し「類似度×評価点」を算出
    rating_matrix_user = rating_matrix_item[:, user_id - 1]
    pred_rating_user = similarity_matrix * rating_matrix_user
    # アイテム（行）ごとに「類似度×評価点」を合計
    pred_rating_user = pred_rating_user.sum(axis=1)

    # ユーザが既に評価したアイテムのスコアはゼロに直す
    pred_rating_user_item = pred_rating_user * rating_matrix_train[:,user_id - 1]

    #ここからレコメンドされたアイテムがどれだけあっていたかを評価していく
    recommend_list = np.argsort(pred_rating_user_item)[::-1][:10] + 1
    purchase_list_user = u_data_test[u_data_test.user_id == user_id].loc[:, 'item_id'].unique()
    for item_id in recommend_list:
        if item_id in purchase_list_user:
            hits += 1
    pre = hits / 10.0
    precision_list.append(pre)


100%|██████████| 943/943 [00:31<00:00, 29.70it/s]


In [ ]:
# 全体の精度検証
precision = sum(precision_list) / len(precision_list)
print('Precision:', precision)

Precision: 0.15726405090137793
